In [7]:
# import Pkg 
# Pkg.add("DelimitedFiles")
# Pkg.add("Random")
# Pkg.add("Distributions")
# Pkg.add("Statistics")
# Pkg.add("SpecialFunctions")
# Pkg.add("QuadGK")
# Pkg.add("DataStructures")
# Pkg.add("DelimitedFiles")
# Pkg.add("Roots")
# Pkg.add("Printf")

   Updating registry at `C:\Users\WeNieds\.julia\registries\General`
  Resolving package versions...
Updating `C:\Users\WeNieds\.julia\environments\v1.5\Project.toml`
  [de0858da] + Printf
No Changes to `C:\Users\WeNieds\.julia\environments\v1.5\Manifest.toml`


In [1]:
using Distributed 
addprocs(23)
@everywhere using DelimitedFiles
@everywhere using Random, Distributions;  Random.seed!();
@everywhere using Distributed

┌ Info: Precompiling Distributions [31c24e10-a181-5473-b8eb-7969acd0382f]
└ @ Base loading.jl:1278


In [9]:
@everywhere using Statistics
@everywhere using SpecialFunctions
@everywhere using QuadGK
@everywhere using DataStructures
@everywhere using DelimitedFiles
@everywhere using Roots
@everywhere using Printf

In [4]:
#SIk
@everywhere function SIk(x,parms)
    (S,I) = x
    (μ,β,κ,γ,N) = parms
    infection = β*(N-I)*I/N + κ*(N-I)
    recov_death = (γ+μ)*I
    event_weight = [infection recov_death]
    event_shift = [[0 1]; [0 -1]]
    return event_weight, event_shift
end

In [5]:
@everywhere function GSP(F::Function,state,parms,endcondition,time_end)
    time_points = collect(0:2:floor(time_end))
    I_vec_out = zeros(length(time_points))
    I_vec_out[1] = state[2]
    t = 0 #initial time is zero
    a = []    
    enact_event = zeros(0)
 
    while t<(time_points[end] +2)
        
        # to build time interval
        t1 = t
        
        ###############
        ## time step ##
        ###############
        a = F(state,parms)

        cumul_a = cumsum(a[1][1,:])
        num_o_events = length(a[1][1,:])
        a0 = cumul_a[num_o_events]
        #random numbers
        r1 = rand()
        r2 = rand()
        dtau = (1/a0)*log(1/r1)
        t += dtau
        
        ################################
        ### Append the output vector ###
        ################################

        timeloc =  findall(x->(x>t1 && x<=t), time_points)
        I_vec_out[timeloc] = repeat([state[2]],inner=1,outer=length(timeloc))
        

        ##############################
        ### determine event ##########
        ##############################
        marker = r2*a0
        i=1
        while i<=num_o_events
            if marker<=cumul_a[i]
                enact_event = i
                i = num_o_events+1
            end
            i+=1
        end


        #######################
        ### change the state ##
        #######################
        state = state + (a[2][enact_event,:])' #these are the state alterations defined by the function


    end #end while
    return time_points, I_vec_out
end 

In [10]:
###################
### run GSP sim ###
###################
@everywhere function run_GSP(kval_u,Nval_u,Bval_u)
    
    # set parameters find equilibrium
    param=[5e-5 Bval_u kval_u 1 Nval_u]

    function i_equation_u(I,param)
        (μ,β,κ,γ,N) = param
        return (κ+β*I)*μ - (μ+κ+β*I)*(μ+γ)*I
    end
    function i_equation(I)
        return i_equation_u(I,param)
    end
    ieq = fzero(i_equation,1)
    function sequilibrium(param)
        (μ,β,κ,γ,N) = param
        return μ / (μ+κ+β*ieq)
    end
    seq = sequilibrium(param)

    # Make initial conditions
    (μ,β,κ,γ,N) = param
    u0 = [floor(seq*param[5]) floor(ieq*param[5])]

    # run the Gillespie simulation
    simdat = GSP(SIk,u0,param,20000,200);
    n_len = length(simdat[1])
    kap_vec = zeros(n_len)
    kap_vec[1] = κ
    N_vec = zeros(n_len)
    N_vec[1] = N
    B_vec = zeros(n_len)
    B_vec[1] = β
    
    # Make strings for file names
    ku = @sprintf("%.3E", κ)
    ku = replace(ku, "." => "p")
    ku = replace(ku, "-" => "m")
    ku = replace(ku, "+" => "")
    
    Nu = @sprintf("%.3E", N)
    Nu = replace(Nu, "." => "p")
    Nu = replace(Nu, "-" => "m")
    Nu = replace(Nu, "+" => "")
    
    Bu = @sprintf("%.3E", β)
    Bu = replace(Bu, "." => "p")
    Bu = replace(Bu, "-" => "m")
    Bu = replace(Bu, "+" => "")
    
    simdat_util = [simdat[1] simdat[2] kap_vec N_vec B_vec] 
    flname = "SIk_keq_"*ku*"_Neq_"*Nu*"_Beq_"*Bu
    writedlm(flname,  simdat_util, ',') 

    return nothing
end

In [11]:
# make sample list of kappa values
kappa_list_util = 10.0 .^(range(-7,stop=-4,length=50))
kappa_list = zeros(0)

# # make sample list of N values
# N_list_util = 10.0 .^(range(2,stop=5,length=30))
# N_list = zeros(0)

# make sample list of N values
#     Example with only one N value
N_list_util = 10.0 .^ 4
N_list = zeros(0)

# make sample list of beta values
B_list_util = range(0.01,stop=1.5,length=50)
B_list = zeros(0)

for i in kappa_list_util
    for j in N_list_util
        for l in B_list_util            
            # make lists for pmap
            append!(kappa_list, i)
            append!(N_list, j)
            append!(B_list,l)
        end
    end
end
pmap(run_GSP,kappa_list,N_list,B_list)

2500-element Array{Nothing,1}:
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 ⋮
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing